In [1]:
# install 3 libraries
!pip install beautifulsoup4 requests selenium
!pip install pyspellchecker

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.6/9.6 MB 33.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 476.0/476.0 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 41.0 MB/s eta 0:00:00


In [ ]:
#importing library
import pandas as pd
import requests
from bs4 import BeautifulSoup
import numpy as np
from google.colab import files
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
nltk.download('stopwords')
import re
import string
nltk.download('punkt')
from spellchecker import SpellChecker
from nltk.tokenize import sent_tokenize

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [3]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [4]:
# importing the input file into a dataframe
df = pd.read_csv('https://docs.google.com/spreadsheets/d/1D7QkDHxUSKnQhR--q0BAwKMxQlUyoJTQ/export?format=csv')


In [5]:
# The urls that needs to be scrapped
urls =df['URL']
urls


,URL
0,https://insights.blackcoffer.com/ml-and-ai-bas...
1,https://insights.blackcoffer.com/streamlined-i...
2,https://insights.blackcoffer.com/efficient-dat...
3,https://insights.blackcoffer.com/effective-man...
4,https://insights.blackcoffer.com/streamlined-t...
...,...
142,https://insights.blackcoffer.com/population-an...
143,https://insights.blackcoffer.com/google-lsa-ap...
144,https://insights.blackcoffer.com/healthcare-da...
145,https://insights.blackcoffer.com/budget-sales-...


In [6]:
# function to scrap the  websites

def scrape_website(url):


  response = requests.get(url) # fetching the html content of each url
  soup = BeautifulSoup(response.content, 'html.parser') # parsing the html content
  article_title = (soup.find('h1')).text.strip() # fetching the title of the article and removing trailing and leading spaces
  article_content = (soup.find('div',class_='td-post-content tagdiv-type')).text.strip()  # fetching the content of the article
  article = article_title + ' ' + article_content  #concatenate the title and the content
  article = re.sub(r'\s+', ' ', article) # remove extra spaces
  return article




### *Preparing for Sentimental Analysis*

In [7]:

# function to load stop words from google drive and exclude the string part after '|'

stop_words = set() #empty set

def load_stop_words(url):

   response = requests.get(url)
   if response.status_code == 200:# if the fetching is successful

    stop_words.update(x.split('|')[0].strip() for x in response.text.splitlines())
   else:
     response.encoding = 'latin-1'
     stop_words.update(x.split('|')[0].strip() for x in response.text.splitlines())


# thuus, stop_words contains all the stop words.

In [8]:
# load the stop words
load_stop_words('https://drive.google.com/uc?export=download&id=1aWxyJI0d9MOk59OZ_unfBY5E-Nvg_ezW')#auditor
load_stop_words('https://drive.google.com/uc?export=download&id=1K-6MjPq5AQg4ICYY6PDfapB7JECUnryD')#currencies
load_stop_words('https://drive.google.com/uc?export=download&id=13LXnH6vaJhvY4s2ai_2oW2qwongU_iAI')#datesandnumbers
load_stop_words('https://drive.google.com/uc?export=download&id=1tTDfLXNPxNuUGZXHQkQhW6wPf4Xnivwr')#generic
load_stop_words('https://drive.google.com/uc?export=download&id=1PnZhcsfjBVxnzwa4N6MrLWf6Kuhhjpdk')#generic long
load_stop_words('https://drive.google.com/uc?export=download&id=1mBOuggD8AVNFjr9sprLoD2_6mVWAgRGE')#names
load_stop_words('https://drive.google.com/uc?export=download&id=1RKxMOHzBdLrGuYb7MCJRTKKPwDG9Agbe')#geographic


In [9]:
# Function to create the dictionaries:

# load the words
def create_dictionary(url):
  response = requests.get(url)
  words = set(x.split('|')[0].strip() for x in response.text.splitlines())

 # filtering stop words from the words
  words = words - stop_words

  # but some words might have spelling-mistakes:
  words_corrected = []
  spell =SpellChecker()

  for word in words:
    if any(not char.isalpha() for char in word):  #skip correcting words with special characters and numbers
      words_corrected.append(word)
    else:
      correction = spell.correction(word)
      words_corrected.append(correction if correction is not None else word )

  #creating the dictionary:
  dictionary = {x : True for x in words_corrected} # True in the values because easier to update
  return dictionary




In [10]:
# creating postive words dictionary

positive_dictionary = create_dictionary('https://drive.google.com/uc?export=download&id=1seAj8G42SmfgUUx8lqVDJofm4Tuh2TOT')




In [11]:
# creating negative words dictionary
negative_dictionary = create_dictionary('https://drive.google.com/uc?export=download&id=1qqMwc_-ayS38HEOB97osO_nkIxRkbnvh')

# **1. SENTIMENT ANALYSIS**

In [16]:
# loop to load all the urls and do sentimental analysis on them:

output=[]  # empty list to collect all the output at the end

# List to store the scores for each url
output = []
iteration_count = 0
# loop over each url in the input file:
for url in urls:

  print(url)
  iteration_count += 1
  print(f"Iteration {iteration_count}")

  article_text_str = scrape_website(url)

  # Convert to uppercase, split, and filter stop words
  article_text = [word.lower() for word in article_text_str.split() if word.upper() not in stop_words]

    # Join and tokenize
  article_text = word_tokenize(' '.join(article_text))

    # Analysis of scores
  positive_score = sum(1 for x in article_text if x in positive_dictionary.keys())
  negative_score = sum(1 for x in article_text if x in negative_dictionary.keys())
  polarity_score = (positive_score - negative_score) / ((positive_score + negative_score) + 0.000001)
  subjectivity_score = (positive_score + negative_score) / (len(article_text) + 0.000001)

    #Analysis of readability:

    # sentences in the scrapped article text
  article_sentences = sent_tokenize(article_text_str)

    # num_of_words in the text
  num_of_words = 0
  for sentences  in article_sentences:
      for words in sentences.split():
        if words.upper() not in stop_words: #cleaning stop words
          num_of_words += 1

    # Average Sentence Length = the number of words / the number of sentences:
  avg_sentence_len = num_of_words/len(article_sentences)

    # Percentage of Complex words = the number of complex words / the number of words

  complex_num_of_words =0

  for sentence  in article_sentences:

        for word in sentence.split():
            syllable_count = 0   #reset to zero

            if word.endswith('es') or word.endswith('ed'):
                continue  #skip this word

            for char in word:
                if char in 'aeiouAEIOU':
                    syllable_count += 1
            if  syllable_count > 2 :
                complex_num_of_words += 1


    # Percentage of Complex words = the number of complex words / the number of words

  percentage_complex_words =complex_num_of_words/num_of_words


    #Fog Index = 0.4 * (Average Sentence Length + Percentage of Complex words)

  Fog_index = 0.4*(avg_sentence_len+percentage_complex_words)

    # Average Number of Words Per Sentence = the total number of words / the total number of sentences

  avg_sentence_len

    # word count

    # Load English stopwords
  stop_words = set(stopwords.words('english'))

    # Tokenize the string
  words = word_tokenize(article_text_str.lower())

    # Remove punctuation and filter out stopwords
  cleaned_words = [word for word in words if word not in stop_words and word not in string.punctuation]

    # Count the number of remaining words
  word_count = len(cleaned_words)


    #syllable per word count = number of syllabes in all the words/number of words
  syllable_count = 0
  for word in cleaned_words:
    if word.endswith('es') | word.endswith('ed'):
      continue
    for char in word:
      if char in 'aeiou':
        syllable_count += 1

  syllable_per_word = syllable_count/word_count


    #Personal pronouns count

  personal_pronouns_count =0

  for word in words:
    if word == 'US':
      continue       # skip iterations
    if word in ('I', 'we', 'my', 'ours', 'us') :
      personal_pronouns_count += 1


    # Average Word Length = Sum of the total number of characters in each word/Total number of words
  word_count_3 =0
  char_count = 0
  for word in cleaned_words:
    word_count_3 += 1
    for char in word:
        char_count += 1


  Avg_word_length = char_count/word_count_3






    # Appending the scores and URL as a dictionary to the list
  output.append({
        'URL_ID': df.loc[df['URL'] == url, 'URL_ID'].values[0],
        'URL': url,
        'POSITIVE SCORE': positive_score,
        'NEGATIVE SCORE': negative_score,
        'POLARITY SCORE': polarity_score,
        'SUBJECTIVITY SCORE': subjectivity_score,
        'AVG SENTENCE LENGTH' : avg_sentence_len,
        'PERCENTAGE OF COMPLEX WORDS':percentage_complex_words,
        'FOG INDEX' :Fog_index,
        'AVG NUMBER OF WORDS PER SENTENCE':avg_sentence_len,
        'COMPLEX WORD COUNT':complex_num_of_words,
        'WORD COUNT':word_count,
        'SYLLABLE PER WORD':syllable_per_word,
        'PERSONAL PRONOUNS':personal_pronouns_count,
        'AVG WORD LENGTH':Avg_word_length
    })

# Convert the list of dictionaries to a DataFrame
df_output = pd.DataFrame(output)

# Exporting to an excel file
df_output.to_excel('article_scores.xlsx', index=False)

print("Scores saved to 'article_scores.xlsx'")


https://insights.blackcoffer.com/ml-and-ai-based-insurance-premium-model-to-predict-premium-to-be-charged-by-the-insurance-company/
Iteration 1
https://insights.blackcoffer.com/streamlined-integration-interactive-brokers-api-with-python-for-desktop-trading-application/
Iteration 2
https://insights.blackcoffer.com/efficient-data-integration-and-user-friendly-interface-development-navigating-challenges-in-web-application-deployment/
Iteration 3
https://insights.blackcoffer.com/effective-management-of-social-media-data-extraction-strategies-for-authentication-security-and-reliability/
Iteration 4
https://insights.blackcoffer.com/streamlined-trading-operations-interface-for-metatrader-4-empowering-efficient-management-and-monitoring/
Iteration 5
https://insights.blackcoffer.com/efficient-aws-infrastructure-setup-and-management-addressing-security-scalability-and-compliance/
Iteration 6
https://insights.blackcoffer.com/streamlined-equity-waterfall-calculation-and-deal-management-system/
Ite

,URL_ID,URL
0,bctech2011,https://insights.blackcoffer.com/ml-and-ai-bas...
1,bctech2012,https://insights.blackcoffer.com/streamlined-i...
2,bctech2013,https://insights.blackcoffer.com/efficient-dat...
3,bctech2014,https://insights.blackcoffer.com/effective-man...
4,bctech2015,https://insights.blackcoffer.com/streamlined-t...


In [17]:
# to download the file;
from google.colab import files
files.download('article_scores.xlsx')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>